In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
words[:10]

In [ ]:
len(words)

In [ ]:
min(len(w) for w in words)

In [ ]:
max(len(w) for w in words)

# Part 1
## We'll start with building **bigram LM**
    - Simple & weak LM

In [ ]:
# Getting individual character bigrams

for w in words[:3]:
    chs = ['<S>'] + list(w) + ['<E>']
#     for ch1, ch2 in zip(w, w[1:]):
    for ch1, ch2 in zip(chs, chs[1:]):
        print(ch1, ch2)

In [ ]:
w

In [ ]:
w[1:]

In [ ]:
list(w)

In [ ]:
['<S>'] + list(w) + ['<E>']

- In order to learn the statistics of which characters are likely to follow other characters:
    - Simplest way is to do by counting (in **bigram LM**)
        - How often any of these combination occurs in the dataset?
        
- We can think of a dictionary, which can maintain counts of bigrams.    

In [ ]:
b = {}

for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1
        # .get returns 0 if the key does not exist in dict
        
#         print(ch1, ch2)

In [ ]:
b

In [ ]:
b.items()
# return tuples of (key, value)

- Now we want to sort the above tuples of (key, value)
- But by default ```sorted``` sorts on 1st item of tuple.
    - Whereas, we want to sort them by values, which are 2nd item of tuples.

In [ ]:
sorted(b.items(), key = lambda kv: -kv[1])
# this lambda takes (key, value) tuple and returns the value

- Now it would be better to keep this information in a 2-d array instead of python dict.
    - **rows** would be the 1st character
    - **columns** would be the 2nd character, following 1st.
    - the **entries** will give us the count of occurence of 1st character followed by 2nd.

In [ ]:
import torch

In [ ]:
# Initializing a tensor with int32 type, for 28x28 size
# 26 alphabets + 2 special characters

N = torch.zeros((28,28), dtype=torch.int32)

- We have characters that are strings,
- But to build the 2-d array we might need to convert them to numbers
- To do so, we would require a lookup table - **from characters to integers**.

In [ ]:
''.join(words)

In [ ]:
set(''.join(words))

In [ ]:
chars = sorted(list(set(''.join(words))))

stoi = {s:i for i,s in enumerate(chars)} # mapping
stoi['<S>'] = 26
stoi['<E>'] = 27

In [ ]:
stoi

In [ ]:
for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    
    for ch1, ch2 in zip(chs, chs[1:]):
        # Map both ch1 & ch2 to their integers
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
N

In [ ]:
N.shape

### Visualizing the array 'N'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(N)

#### Even better visualization

In [ ]:
itos = {i:s for s,i in stoi.items()}

In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(N, cmap="Blues")
for i in range(28):
    for j in range(28):
        chstr = itos[i] + itos[j] # character string - bigrams in character representation
        plt.text(j, i, chstr, ha='center', va='bottom', color='gray') # plot the bigram text
        plt.text(j, i, N[i, j].item(), ha='center', va='top', color='gray') # plot the no. of times a bigram occurs

plt.axis('off');

#### In the above viusal we see - char in row followed by char in column with their counts
    - Some of them occur often, and some do not
    - Consider the last row - end of word **<E>** followed by a character does not make sense
        - Also shown by 0s in the entire row
    - Consider the 2nd last column - start of a word **<S>** following a character does not make sense
        - Also shown by all 0s in the entire column
    - Another unintuitive thing - looking at last 2x2 matrix, you see <S> & <E> occuring together in any order does not make sense.
    
    - Using <S> and <E> makes the visual crowded.
    
**Therefore we make some changes considering the above points**

In [ ]:
N = torch.zeros((27,27), dtype=torch.int32) # We're going to have only 1 special token

In [ ]:
chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i,s in enumerate(chars)} # mapping
stoi['.'] = 0 # The only special character, but we move it to 0, just to make it pleasing; And offset remaining char
# stoi['<E>'] = 27

itos = {i:s for s,i in stoi.items()}

In [ ]:
stoi

In [ ]:
itos

In [ ]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    
    for ch1, ch2 in zip(chs, chs[1:]):
        # Map both ch1 & ch2 to their integers
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(N, cmap="Blues")
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j] # character string - bigrams in character representation
        plt.text(j, i, chstr, ha='center', va='bottom', color='gray') # plot the bigram text
        plt.text(j, i, N[i, j].item(), ha='center', va='top', color='gray') # plot the no. of times a bigram occurs

plt.axis('off');

# Counts array of our dataset

In [ ]:
# Looking at how often a character is starting the name word
N[0, :]

In [ ]:
p = N[0].float()
p = p/p.sum() #Normalized and converted to probabilities
p #Probability of any character to be the first character in a word/name

In [ ]:
sum(p)

- Now we sample from above distribution
    - using pytorch **multinomial**; you give me prob - I'll give you integers (which are sampled according to the distribution)
    - pytorch **generator**; to make everything deterministic

############################--------------------------################################

In [ ]:
g = torch.Generator().manual_seed(2147483647)

p = torch.rand(3, generator=g)
p

In [ ]:
p = p / p.sum()
p

- Now we use ```torch.multinomial``` to draw sample from above distribution

In [ ]:
torch.multinomial(p, num_samples=20, replacement=True, generator=g)

# What this will do is give us numbers 0,1,2 in ratio that is seen in "p" distribution
# Around 60% of num_samples will be 0, 30% will be 1, and only 10% are 2.

############################--------------------------################################

In [ ]:
# Lets get back to our original p distribution
p = N[0].float()
p = p/p.sum()
p

In [ ]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
# We've sampled the 1st character of our word
ix, itos[ix]

#### As you can infer from above visual, 'm' starts 2538 words out of total 32000 words (approx 10% of the total)

- **m** is already sampled, we now proceed with further characters
- To draw next, we jump to the row of **m**, and sample from there.

In [ ]:
# So now we write a loop to sample word

g = torch.Generator().manual_seed(2147483647)

for i in range(20):
    ix = 0
    out = []
    while True:
        p = N[ix].float()
        p = p/p.sum()

        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

#### Obviously, the names above hardly sound name-like
- Probably because the **bi-gram** model is very weak
    - Also, it only knows what is 1 step ahead of it while generating, doesn't matter it it's the first word or not

#### To justify if it's at least doing something sensible generation:
    - We try to generate with equaly likely distribution (uniform)
    - This does even worse

In [ ]:
# So now we write a loop to sample word

g = torch.Generator().manual_seed(2147483647)

for i in range(20):
    ix = 0
    out = []
    while True:
#         p = N[ix].float()
#         p = p/p.sum()
        p = torch.ones(27) / 27.0

        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

## Removing in-efficiency

In [ ]:
N[0]

In [ ]:
# Broadcasting
# P:                           27, 27
# P.sum(axis=1, keepdim=True): 27,  1

# eligible for broadcasting

In [ ]:
P = N.float()

In [ ]:
P.sum(axis=1, keepdim=True).shape

In [ ]:
# We want to convert entire "N" tensor into float; and divide every row elements by its sum (sum of ROW)
P /= P.sum(axis=1, keepdim=True) #Denominator gets stretched from 1 to 27 columns; with BROADCASTING; and then do elementwise division
P.shape

In [ ]:
P[0]

In [ ]:
P[0].sum()

In [ ]:
# So now we write a loop to sample word

g = torch.Generator().manual_seed(2147483647)

for i in range(20):
    ix = 0
    out = []
    while True:
        p = P[ix]
        
        # These two lines of code below are very inefficient, doing same operation everytime
#         p = N[ix].float()
#         p = p/p.sum()

        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

## Something SCARY about broadcasting

In [ ]:
P = N.float()
P[0]

In [ ]:
P.sum(1, keepdim=True).shape

In [ ]:
P.sum(1, keepdim=True)

In [ ]:
P.sum(1).shape

In [ ]:
P.sum(1)

In [ ]:
# Broadcasting
# P:                           27, 27
# P.sum(axis=1):                   27

# Broadcast will create an axis in 1 position, and will even do elementwise division,
# But this is a BUG

In [ ]:
P = P / P.sum(axis=1)

In [ ]:
P[0].sum() # This is wrong, it went on to normalize the columns instead of rows

In [ ]:
P[:, 0].sum() # Wrongly, columns got normalized

#### Scary part ends ^

## Evaluating the bi-gram model quality - loss function

In [ ]:
# So now we write a loop to sample word

g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    ix = 0
    out = []
    while True:
        p = P[ix]
        
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

In [ ]:
# GOAL: maximize likelihood of the data w.r.t. model parameters (statistical modeling)
# equivalent to maximizing the log likelihood (because log is monotonic)
# equivalent to minimizing the negative log likelihood
# equivalent to minimizing the average negative log likelihood

# log(a*b*c) = log(a) + log(b) + log(c)

In [ ]:
log_likelihood = 0.0
n = 0

for w in words[:3]:
    chs = ['.'] + list(w) + ['.']
    
    for ch1, ch2 in zip(chs, chs[1:]):
        # Map both ch1 & ch2 to their integers
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

# In case of all equally likely probabilities, a probability of ~4% (1/27) would be called untrained.
# But as you see in prob printed here, some are as high as 40%,
# we can say our bi-gram model has certainly learned something.

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

In [ ]:
# Not how do we use above shown probabilities, to judge quality of the model

In [ ]:
# Evaluate the average nll for entire dataset

log_likelihood = 0.0
n = 0

for w in words:
    chs = ['.'] + list(w) + ['.']
    
    for ch1, ch2 in zip(chs, chs[1:]):
        # Map both ch1 & ch2 to their integers
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
#         print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

# In case of all equally likely probabilities, a probability of ~4% (1/27) would be called untrained.
# But as you see in prob printed here, some are as high as 40%,
# we can say our bi-gram model has certainly learned something.

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

In [ ]:
# Finding nll for a specific word - "mrigank"

log_likelihood = 0.0
n = 0

for w in ['mrigank']:
    chs = ['.'] + list(w) + ['.']
    
    for ch1, ch2 in zip(chs, chs[1:]):
        # Map both ch1 & ch2 to their integers
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

# In case of all equally likely probabilities, a probability of ~4% (1/27) would be called untrained.
# But as you see in prob printed here, some are as high as 40%,
# we can say our bi-gram model has certainly learned something.

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

# Average nll for "mrigank" = 2.98

In [ ]:
# But what if we add add a 'g' in the end

log_likelihood = 0.0
n = 0

for w in ['mrigankg']:
    chs = ['.'] + list(w) + ['.']
    
    for ch1, ch2 in zip(chs, chs[1:]):
        # Map both ch1 & ch2 to their integers
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

# In case of all equally likely probabilities, a probability of ~4% (1/27) would be called untrained.
# But as you see in prob printed here, some are as high as 40%,
# we can say our bi-gram model has certainly learned something.

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

# Average nll for "mrigank" = inf

### The average nll becomes - inf
    - Reason: count of the bigram ```kg``` is 0
    - This can be verified from the last visualization

## To handle above problem - we do Model Smoothing
    - 1 is added to all numbers in matrix N
    - This is called - adding some fake counts

In [ ]:
P = (N+1).float()
P /= P.sum(1, keepdim=True)

In [ ]:
log_likelihood = 0.0
n = 0

for w in ['mrigankg']:
    chs = ['.'] + list(w) + ['.']
    
    for ch1, ch2 in zip(chs, chs[1:]):
        # Map both ch1 & ch2 to their integers
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

# In case of all equally likely probabilities, a probability of ~4% (1/27) would be called untrained.
# But as you see in prob printed here, some are as high as 40%,
# we can say our bi-gram model has certainly learned something.

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

# Its not a zero any longer

In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(N+1, cmap="Blues")
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j] # character string - bigrams in character representation
        plt.text(j, i, chstr, ha='center', va='bottom', color='gray') # plot the bigram text
        plt.text(j, i, (N+1)[i, j].item(), ha='center', va='top', color='gray') # plot the no. of times a bigram occurs

plt.axis('off');

# Counts array of our dataset

In [ ]:
# One last time, sampling characters for word

g = torch.Generator().manual_seed(2147483647)

for i in range(20):
    ix = 0
    out = []
    while True:
        p = P[ix] #P is smoothened here
        
        # These two lines of code below are very inefficient, doing same operation everytime
#         p = N[ix].float()
#         p = p/p.sum()

        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

####################################---------------------------------------######################################

# Part 2
## We build neural network
    - Given the 1st character of bigram, we predict next character

In [ ]:
# Creating the training set of bigrams (x, y)

xs, ys = [], []

for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        print(ch1, ch2)
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs) # torch.Tensor would give float values
ys = torch.tensor(ys)

In [ ]:
xs

In [ ]:
ys

#### We should not directly feed in these integers to the network
    - Simplest way of encoding them is One Hot encoding

In [ ]:
import torch.nn.functional as F

In [ ]:
xenc = F.one_hot(xs, num_classes=27).float()

In [ ]:
xenc.shape

In [ ]:
plt.imshow(xenc)

In [ ]:
# Weights initialization

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=g)

In [ ]:
xenc @ W      # matrix multiplication operator

In [ ]:
(xenc @ W)[3, 13]

In [ ]:
xenc[3]

In [ ]:
W[:, 13]

In [ ]:
(xenc[3] * W[:, 13]).sum()

In [ ]:
logits = xenc @ W # log-counts

# These 2 steps combine to make "SoftMax"
counts = logits.exp() # equivalent to the N matrix
probs = counts / counts.sum(1, keepdim=True)

In [ ]:
probs

In [ ]:
# probs[0]

# The 1st character (".") of 1st bigram (".e"), is converted to "int" and encoded with one-hot
# then fed into neural network
# Out came a distribution of 27 (# of neurons) probabilities

In [ ]:
probs.shape

### Now we have to optimize "W", such that probabilities coming out are pretty good
    - To do so, we use "loss function"

In [ ]:
nlls = torch.zeros(5)
for i in range(5):
    # i-th bigram
    x = xs[i].item() # input character index
    y = ys[i].item() # label character index
    
    print('----------------')
    print(f'bigram example {i+1}: {itos[x]}{itos[y]} indexes ({x},{y})')
    print('input to neural nets:', x)
    print('output probabilities form neural nets:', probs[i])
    print('label (actual next character):', y)
    
    p = probs[i,y]
    print('probability assigned by the net to the correct character:', p.item())
    
    logp = torch.log(p)
    print('log likelihood: ', logp)
    
    nll = -logp
    print('negative log likelihood:', nll)
    nlls[i] = nll
    
print('\n================\n')
print('Average nll, i.e. loss =', nlls.mean().item())

### Now, we have to minimize this loss by tuning W with backprop

In [ ]:
# ----------------------- OPTIMIZATION !!  GRADIENT DESCENT     ------------------------------

In [ ]:
xs

In [ ]:
ys

In [ ]:
# Weights initialization - random

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=g, requires_grad=True)

In [ ]:
# forward pass

xenc = F.one_hot(xs, num_classes=27).float()
logits = xenc @ W # log-counts

# These 2 steps combine to make "SoftMax"
counts = logits.exp() # equivalent to the N matrix
probs = counts / counts.sum(1, keepdim=True)


# loss calculation

# We would want to extract probabilities of correct index from "probs"; but below is inefficient way
# probs[0,5], probs[1,13], probs[2,13], probs[3,1], probs[4,0]

# efficient way
# probs[torch.arange(5), ys]

# Average - negative - log - likelihood
loss = -probs[torch.arange(5), ys].log().mean()
loss

In [ ]:
# backward pass

W.grad = None # set the grads to 0
# similar to doing W.grad = 0

loss.backward()
# this fills in the gradients of all intermediate steps in the computational graph


In [ ]:
# W.grad

# every element here tells us:
# influence of that weight on loss
# If +ve: means it will nudge the loss positive direction if increased.
# If -ve: nudge the loss lower

In [ ]:
W.grad.shape, W.shape

In [ ]:
# weight update
W.data += -0.1 * W.grad

In [ ]:
# ----------------------------- EVERYTHING TOGETHER -------------------------------------------

In [ ]:
xs, ys = [], []

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs) # torch.Tensor would give float values
ys = torch.tensor(ys)
num = xs.nelement()
print("number of examples: ", num)

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=g, requires_grad=True)

In [ ]:
# gradient descent

for k in range(100):
    
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W # log-counts
    counts = logits.exp() # equivalent to the N matrix
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(num), ys].log().mean()
    print("loss: ", loss.item())
    
    # backward pass
    W.grad = None # set the grads to 0
    loss.backward()
    
    # update weights
    W.data += -50 * W.grad

## Few notes: on smoothing

1. In bi-gram approach, smoothing was done by adding a number (1) to the counts in N matrix.
    - Instead if we would have added a very large number (say 1,000,000), all counts would have become relatively equal.
    - And every bigram would have become equally likely.
    - Uniform distribution

2. Gradient based framework - has an equivalent to smoothing
    - Suppose, if **W** was initialized as all 0.
        - All **logits** would have become 0.
            - All **counts** would have become 1.
                - **probs** would have turned out to be **uniform**

    - Trying to incentivizing **W** near 0;
        - Is equivalent to label smoothing
            - The more you incentivize that, more **smooth** distribution you'll achieve

## Few notes: on Regularization

1. We can augment the loss function
    - To have a small component called **regularization loss**

In [ ]:
(W ** 2).mean()

In [ ]:
# gradient descent

for k in range(100):
    
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W # log-counts
    counts = logits.exp() # equivalent to the N matrix
    probs = counts / counts.sum(1, keepdim=True)
    
    # Adding this regularization component - pushes optimization towards zeroing of the W
    # Adding larger number to bi-gram approach - relates to increasing 0.001 term below
    # The more you increase 0.001, the regualrization term dominates the 1st part
    # ==> The more W weights will be unable to grow, so everything will be kind of uniform prediction
    loss = -probs[torch.arange(num), ys].log().mean() + (0.001 * (W ** 2).mean())
    print("loss: ", loss.item())
    
    # backward pass
    W.grad = None # set the grads to 0
    loss.backward()
    
    # update weights
    W.data += -50 * W.grad

### Sampling form Neural Net

In [ ]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    ix = 0
    out = []
    while True:
        # Before
        p = P[ix] #P is smoothened here
        
        # Now
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W # log-counts
        counts = logits.exp() # equivalent to the N matrix
        probs = counts / counts.sum(1, keepdims=True)
        
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

In [ ]:
# These names generations are exactly same as one from bi-gram model
# same loss
# W is equivalent to log-counts; analogous to the N matrix
# 